In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import os
import re
import pandas as pd

# URL of the page containing links to the scripts
main_url = 'https://www.seinfeldscripts.com/seinfeld-scripts.html'

# Headers to mimic a standard web browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

# Fetch the main page with custom headers
response = requests.get(main_url, headers=headers)
html = response.text

# Parse the HTML
soup = BeautifulSoup(html, 'html.parser')

# Find all links and correct them if needed
script_links = ['https://www.seinfeldscripts.com/' + link['href'].strip() for link in soup.select('table a[href]')]

# Remove duplicate links to handle unique episode cases
script_links = list(dict.fromkeys(script_links))

# Directory for CSV
os.makedirs('Seinfeld_Scripts', exist_ok=True)
csv_file_path = 'Seinfeld_Scripts/Seinfeld_Scripts.csv'

# Define the episode ranges for each season
season_ranges = {
    1: range(1, 6),   # Episodes 1-5
    2: range(6, 18),  # Episodes 6-17
    3: range(18, 41), # Episodes 18-40
    4: range(41, 65), # Episodes 41-64
    5: range(65, 87), # Episodes 65-86
    6: range(87, 111), # Episodes 87-110
    7: range(111, 135), # Episodes 111-134
    8: range(135, 157), # Episodes 135-156
    9: range(157, 181)  # Episodes 157-180
}

# Function to determine season and episode number
def get_season_and_episode(index):
    for season, episodes in season_ranges.items():
        if index in episodes:
            return season, index - episodes.start + 1
    return None, None  # Shouldn't happen if ranges are defined correctly

# Open a CSV file to save the extracted data
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Character', 'Dialogue', 'EpisodeNo', 'SEID', 'Season']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Process each script link
    for index, script_url in enumerate(script_links, start=1):
        script_response = requests.get(script_url, headers=headers)
        script_html = script_response.text
        script_soup = BeautifulSoup(script_html, 'html.parser')
        script_text = script_soup.find('div', id='content').get_text(separator='\n')

        season, episode_no = get_season_and_episode(index)
        seid = f'S{season:02}E{episode_no:02}'

        # Adjust parsing for dialogues
        lines = script_text.split('\n')
        for line in lines:
            line = line.strip()
            if not line.startswith('[') and ':' in line:  # Skip scene descriptions
                match = re.match(r'^([A-Z]+(?: [A-Z]+)*):(.+)', line)
                if match:
                    character, dialogue = match.groups()
                    writer.writerow({
                        'Character': character,
                        'Dialogue': dialogue.strip(),
                        'EpisodeNo': float(episode_no),
                        'SEID': seid,
                        'Season': float(season)
                    })

        print(f'Processed script: {seid} - {script_url}')

# Load CSV, adjust the index column, and save again without the header for the index column
df = pd.read_csv(csv_file_path)
df.to_csv(csv_file_path, index_label='', index=True)

# Inform user of completion
print("All scripts have been processed and saved to CSV.")


Processed script: S01E01 - https://www.seinfeldscripts.com/TheSeinfeldChronicles.htm
Processed script: S01E02 - https://www.seinfeldscripts.com/TheStakeout.htm
Processed script: S01E03 - https://www.seinfeldscripts.com/TheRobbery.htm
Processed script: S01E04 - https://www.seinfeldscripts.com/MaleUnbonding.htm
Processed script: S01E05 - https://www.seinfeldscripts.com/TheStockTip.htm
Processed script: S02E01 - https://www.seinfeldscripts.com/TheExGirlfriend.htm
Processed script: S02E02 - https://www.seinfeldscripts.com/ThePonyRemark.htm
Processed script: S02E03 - https://www.seinfeldscripts.com/TheJacket.htm
Processed script: S02E04 - https://www.seinfeldscripts.com/ThePhoneMessage.htm
Processed script: S02E05 - https://www.seinfeldscripts.com/TheApartment.htm
Processed script: S02E06 - https://www.seinfeldscripts.com/TheStatue.htm
Processed script: S02E07 - https://www.seinfeldscripts.com/TheRevenge.htm
Processed script: S02E08 - https://www.seinfeldscripts.com/TheHeartAttack.htm
Proce

In [ ]:
import pandas as pd

# Path to the CSV file
csv_file_path = 'Seinfeld_Scripts/Seinfeld_Scripts.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Specify the column name you want to check for unique entries
column_name = 'Character'  # Change this to the name of the column you're interested in

# Get unique entries in the specified column
unique_entries = df[column_name].unique()

# Print the unique entries
print(f"Unique entries in the '{column_name}' column:")
for entry in unique_entries:
    print(entry)

# Optionally, print the number of unique entries
print(f"Total number of unique entries in '{column_name}': {len(unique_entries)}")

Unique entries in the 'Character' column:
JERRY
GEORGE
WAITRESS
CLAIRE
KRAMER
LAURA
ELAINE
POLICEMAN
BOTH
CAROL
LARRY
DIANE
RIGHT MAN
LEFT MAN
RIGHT GUY
JOEL
JERRY AND JOEL
CUSTOMER
GOERGE
TELLER
JERRY AND ELAINE
VANESSA
STOCK BOY
DRY CLEANER
MARLENE
RECEPTIONIST
HELEN
MORTY
LEO
MANYA
ISAAC
MAN
INTERN
SALESMAN
ALTON
BENES
MANAGER
HAROLD
MANNY
ROXANNE
RITA
JOANNE
STAN
SUSIE
JANICE
RAVA
RAY
ELAINE AND RAY
JERRY AND GEORGE
GEORGGE
LEVITAN
VIC
GLENDA
DAN
BILL
AVA
NEWMAN
TV VOICE
DOCTOR
TOR
DRIVER
ASSISTANT
JERRY AND KRAMER
MAN IN NEIGHBORING BED
TABACHNICK
STEWARDESS
LESLIE
WOMAN
MARY
MARRY
ANTONIO
ELAIEN
ED
PATRICE
FD
TINA
ALL
DORIS
JACK
EVELYN
PHOTOGRAPHER
STELLA
MAN IN THE CROWD
WOMAN IN THE CROWD
CHIROPRACTOR
READER
LIBRARIAN
SECRETARY
LIPPMAN
HEYMAN
BOOKMAN
MARION
SH
MARIAN
FILE
CUT TO
TO
FG
JS
MOLLIKA
NOEL
N
STEVE
GUY
OTHER GUY
ROBERTA
OLD GUY
RICHIE
GINA
MARTIN
RULA
W
GEROGE
KEITH
CABBY
MIKE
BABY
CARRIE
TALL GIRL
NINA
USHER
WEST
TVVOICE
GIRLFRIEND
BIKER
CANDICE BERGEN
STU
JAY
DEVOLA